In [ ]:
import os
import re
import sys
import glob
import pickle
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

import dtw
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas, load_data_slide
from dlml.nn import predict

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = [3]
rec_bus_IDs = [3, 14, 17]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory']

experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=False)
model.compile()
data_dirs = ['..' + os.path.sep +
             os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d])
             for data_dir in network_parameters['data_dirs']]
for i in range(len(data_dirs)):
#              f'/H_G1_{H_G1}/stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
    if H_G1 is not None:
        data_dirs[i] += f'/H_G1_{H_G1}'
    data_dirs[i] += '/stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names'] if 'bus' in var_name]
var_names_fmt = OrderedDict({k + '{}': [] for k in tmp})
tmp = [re.findall('.*_line', var_name)[0] for var_name in network_parameters['var_names'] if 'line' in var_name]
for k in tmp:
    var_names_fmt[k + '_{}_{}'] = []
var_names_fmt = list(var_names_fmt.keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20)
])


generators_areas_map = {
    'default': [
        ['G02', 'G03'],
        ['G04', 'G05', 'G06', 'G07'],
        ['G08', 'G09', 'G10'],
        ['G01']
    ]
}

P_nom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04':  800e6, 'G05':  300e6,
         'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}


window_dur = 60
window_step = 1

var_names = network_parameters['var_names']
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

In [ ]:
H_values = [
    default_H,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.3), ('G03', 3.3), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 5.8), ('G03', 5.8), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2)
    ])
]
N_H = len(H_values)

measure_exact = []

data_normalized = []
measure = []
area_inertia = []

for H in H_values:
    data_file = data_dir + '/ieee39_PF_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
N_layers = len(model.layers)
N_vars = len(var_names)
submodels = [
    keras.Model(inputs=model.inputs, outputs=[model.layers[i].output for i in range(i,i+N_vars)])
     for i in range(N_vars, N_layers - N_vars - 3, N_vars)
]
for layer in model.layers[-4:]:
    submodels.append(keras.Model(inputs=model.inputs, outputs=layer.output))

## Continue from here

In [ ]:
N_blocks = len(data_sliding[0][var_names[0]])
inputs_to_network = []
inputs_to_classifier = []
outputs = []
for data_slid in data_sliding:
    for i in range(N_blocks):
        x = {var_name: tf.constant(data_slid[var_name][i:i+1,:], dtype=tf.float32) for var_name in var_names}
        inputs_to_network.append(np.concatenate([np.squeeze(data_slid[var_name][i:i+1,:]) for var_name in var_names]))
        inputs_to_classifier.append(np.squeeze(submodels[-3].predict(x)))
#         inputs_to_classifier.append(np.squeeze(submodels[-2].predict(x)))
        outputs.append(np.squeeze(submodels[-1].predict(x)))
inputs_to_network = np.array(inputs_to_network)
inputs_to_classifier = np.array(inputs_to_classifier)
outputs = np.array(outputs)
scaled_inputs_to_network = StandardScaler().fit_transform(inputs_to_network)
scaled_inputs_to_classifier = StandardScaler().fit_transform(inputs_to_classifier)

In [ ]:
area_inertia

In [ ]:
col = 'kr'
fig,ax = plt.subplots(2, 1, figsize=(10,5), sharex=False)
j = 10
for i in range(2):
    ax[0].plot(inputs_to_network[i*N_blocks + j], col[i], lw=1)
    ax[1].plot(inputs_to_classifier[i*N_blocks + j], col[i], lw=1, label=f'{outputs[i*N_blocks+j,0]:.2f}')
ax[1].legend(loc='best')

In [ ]:
inputs_to_classifier[i + j].sum()

In [ ]:
inputs_to_classifier[i*N_blocks + j].sum()

In [ ]:
dst_inputs_to_network = np.zeros(N_blocks)
dst_inputs_to_classifier = np.zeros(N_blocks)
for i in range(N_blocks):
    alignment = dtw.dtw(inputs_to_network[i], inputs_to_network[i+N_blocks], keep_internals=False)
    dst_inputs_to_network[i] = alignment.distance
    alignment = dtw.dtw(inputs_to_classifier[i], inputs_to_classifier[i+N_blocks], keep_internals=False)
    dst_inputs_to_classifier[i] = alignment.distance

In [ ]:
np.percentile(dst_inputs_to_network, [0.25,0.5,0.75])

In [ ]:
np.percentile(dst_inputs_to_classifier, [0.25,0.5,0.75])

In [ ]:
i = 0
m = np.min([inputs_to_network[i].min(), inputs_to_network[i+N_blocks].min()])
M = np.max([inputs_to_network[i].max(), inputs_to_network[i+N_blocks].max()])
dst_inputs_to_network = np.sqrt((inputs_to_network[i] - inputs_to_network[i+N_blocks]) ** 2) / (M - m)
m = np.min([inputs_to_classifier[i].min(), inputs_to_classifier[i+N_blocks].min()])
M = np.max([inputs_to_classifier[i].max(), inputs_to_classifier[i+N_blocks].max()])
dst_inputs_to_classifier = np.sqrt((inputs_to_classifier[i] - inputs_to_classifier[i+N_blocks]) ** 2) / (M - m)

plt.plot(dst_inputs_to_network, 'k', lw=0.5)
plt.plot(dst_inputs_to_classifier, 'r', lw=0.5)

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_network[0] - scaled_inputs_to_network[N]) ** 2))

In [ ]:
np.sqrt(np.sum((scaled_inputs_to_classifier[0] - scaled_inputs_to_classifier[N]) ** 2))

In [ ]:
pca = PCA(n_components = 2)
pc_inputs_to_network = pca.fit_transform(scaled_inputs_to_network)
pc_inputs_to_classifier = pca.fit_transform(scaled_inputs_to_classifier)

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(7,4))
ax[0].plot(pc_inputs_to_network[:N,0], pc_inputs_to_network[:N,1], 'k.')
ax[0].plot(pc_inputs_to_network[N:,0], pc_inputs_to_network[N:,1], 'r.')
ax[1].plot(pc_inputs_to_classifier[:N,0], pc_inputs_to_classifier[:N,1], 'k.')
ax[1].plot(pc_inputs_to_classifier[N:,0], pc_inputs_to_classifier[N:,1], 'r.')

In [ ]:
embedding_inputs_to_network = umap.UMAP().fit_transform(scaled_inputs_to_network)
embedding_inputs_to_classifier = umap.UMAP().fit_transform(scaled_inputs_to_classifier)

In [ ]:
plt.plot(embedding_inputs_to_network[:N,0], embedding_inputs_to_network[:N,1],
         'ko', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_network[N:,0], embedding_inputs_to_network[N:,1],
         'ks', ms=4)
plt.plot(embedding_inputs_to_classifier[:N,0], embedding_inputs_to_classifier[:N,1],
         'ro', ms=4, markerfacecolor='w')
plt.plot(embedding_inputs_to_classifier[N:,0], embedding_inputs_to_classifier[N:,1],
         'rs', ms=4)

In [ ]:
output = outputs[0]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=True)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[1]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[2]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[3]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[4]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
output = outputs[5]
rows = output[0].shape[2] + 1
rows = 8
fig,ax = plt.subplots(rows, 6, figsize=(16,rows), sharex=False)
for j,key in enumerate(x.keys()):
    for i,a in enumerate(ax[:,j]):
        if i == 0:
            a.plot(np.squeeze(x[key].numpy()), 'r', lw=1)
        else:
            a.plot(np.squeeze(output[j][0,:,i-1]), 'k', lw=1)
        a.set_xticks([])
        a.set_yticks([])
        for side in 'right','top':
            a.spines[side].set_visible(False)
        if i == rows:
            break
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10,3))
for i,a in enumerate(ax):
    a.plot(outputs[6+i][0], 'k', lw=1)
    a.set_xticks([])
    a.set_yticks([])
    for side in 'right','top':
        a.spines[side].set_visible(False)
fig.tight_layout()